In [849]:
# # This notebook gets all the file and subfile names from a folder
# # We'll be using this to scrape all the SKUs from a manufacturer's list of products

# import pandas as pd
# import glob
# import os

# df=pd.read_csv('Safewaze-Products-MAR2021-v4-categories.csv')
# df.head()

In [850]:
# image_data = [];
# path = r'C:\Users\robin\48WS-Work\Adding Products Data Scraping\Safewaze-Products-Mar2021\Safewaze-Images'

# for root, dirs, files in os.walk(path):
#     for file in files:
#         if file.endswith('.png') or file.endswith('.jpg'):
#             print(os.path.join(file))
        

In [851]:
# Test to change entire data frame to string - everything is now an object
# https://stackoverflow.com/questions/42676982/python-turn-all-items-in-a-dataframe-to-strings

# import pandas as pd
# import glob
# import os

# df=pd.read_csv('Safewaze-Products-MAR2021-v4-categories.csv')

# df = df.astype(str)
# df.dtypes

In [852]:
import pandas as pd
import glob
import numpy as np
import os

# PLEASE NOTE THAT THE FILE THAT I AM IMPORTING HAS CATEGORIES ADDED FOR ONLY A FEW PRODUCTS
df=pd.read_csv('Safewaze-Products-MAR2021-v4-categories.csv')
df = df.astype(str)
df.dtypes

path = r'C:\Users\robin\48WS-Work\Adding Products Data Scraping\Safewaze-Products-Mar2021\Safewaze-Images'
image_list = []

# Check if a file is an image
def is_image(filename):
    return filename.endswith('.png') or filename.endswith('.jpg') or filename.endswith('.jpeg')

# Create List of Image Files
for root, dirs, files in os.walk(path):
    for file in files:
        if is_image(file):
            image_list.append(file)

for index, row in df.iterrows():
    for image in image_list:
        if df['SKU #'][index] in image:
            df['Image'][index] = image
            

df.head(3)

,48WS.com URL,Category,SKU #,Product Name,Image,Manufacturer_URL,Weight,Manufacturer,Description,Quick Overview,Details,Group Name,Info/Size/Color,UPC
0,nan,nan,9013,9013 Behind The Web Bracket,9013 (Updated Version).png,https://safewaze.com/,nan,Safewaze,The Safewaze 9013 dual leg bracket is designed...,nan,Zinc-Plated Steel \nForged Steel \t\nBracket r...,nan,Materials : Forged Steel / Aluminum Alloy; \nZ...,nan
1,nan,nan,018-4000,Chain Anchor,018-4000.png,https://safewaze.com/,nan,Safewaze,Designed for flat or sloped wood roof applicat...,nan,\nO-ring connection point \nDurable steel cha...,nan,Materials : Steel and Reinforced Synthetic\nWe...,nan
2,nan,nan,018-4001,Beam Trolley Anchor,018-4001.png,https://safewaze.com/,nan,Safewaze,The 018-4001 beam trolley anchor can be quickl...,nan,Lightweight Aluminum Body \nCoated Steel D-Ri...,nan,"Materials : Aluminum, Stainless Steel, Ertalon...",nan


In [853]:
# Test - Checking for Newlines and Other Misc. Styling
df.iloc[2][10] # not counting header row, 3rd row and 7th column (so they both were ZBB)

# How iloc syntax works:
# df.iloc[skip header + ZBB row][ZBB column]

' Lightweight\xa0Aluminum Body \nCoated Steel D-Ring Attachment Point \nAbrasion, Heat & Chemical Resistant\xa0Nylon Wheels \nAdjusts From 3.5”-14” \nFits Beam Flange\xa0up to 1.75” Thick \n310 lb. Weight Capacity'

In [854]:
# Notes: FOR COLUMN 6 ZBB, 'Details':
# If string begins with ["\r\r\n \t], replace it with <ul><li>, and if it ends with [\r\r\n"], replace it with </li></ul>
# Replace all [\r\r\n \t] with [<li> </li>]
# If the string begins with [" \n] or [' \n] or [' \t] or [" \t], replace it with [<ul><li>].
# Otherwise, if it begins with [' ] or [" ] or ['] or ["], replace it with [<ul><li>].
# Replace the final single or double quote in the string with [</li></ul>]
# Replace all [\t\n] (no space before or after) with [</li></li>]
# Replace all [\n] and [\t] (no space before them) with [</li> <li>]. Make sure you get rid of the space before the [ \n], too.
# Replace all [\xa0] with [ ] an empty space

#~~~~~~~~~~~~~~~~~~~~

#Rules that won't work because there are exceptions:
# Replace [" \n]  and [' \n] with <ul><li> to control for the start of the list and the first bulletpoint#

In [855]:
# Testing Details Column
# df.iloc[2][10]
# df.iloc[4][10]
# df.iloc[68][10]
# df.iloc[100][10]
# df.iloc[3][10]
# df.iloc[144][10]
# df.iloc[400][10]
# df.iloc[190][10]
# df.iloc[150][10]

In [856]:
# TESTS FOR THE INFO/SIZE/UPC COLUMN
# df.iloc[5][12]
# df.iloc[20][12]
# df.iloc[150][12]
# df.iloc[333][12]
# df.iloc[379][12]
# df.iloc[47][12]

In [857]:
# Notes: FOR COLUMN 7, 'Info/Size/Color':
# Replace starting single or double quote with <ul><li>
# Replace ending single or double quote with </li></ul>
# Replace [\n] with </li> <li>

In [858]:
# df.replace(
#     {
#         'Details': {
#             r' \n' : '</li></ul>',
#             r'\n' : '</li></ul>',
#             r'<li></li><li>' : '<li></li>',
#             r'\t' : '</li><li>',
#             r'<ul><li></li>' : '<ul><li>',
#             r'<ul><li><ul><li>' : '<ul><li>',
#             'â€²' : '\"',
#             'â€“' : '\-',
#         }
#     }, regex = True
# )

df['Details'] = "<ul><li>" + df['Details'] + "</li></ul>"
df['Details'] = df['Details'].replace(r' \n', r'</li><li>', regex=True)
df['Details'] = df['Details'].replace(r'\n', r'</li><li>', regex=True)
df['Details'] = df['Details'].replace(r' \t', '</li><li>', regex=True)
df['Details'] = df['Details'].replace(r'\t', '</li><li>', regex=True)
df['Details'] = df['Details'].replace(r'<li> </li>', '', regex=True)
df['Details'] = df['Details'].replace(r'<li></li>', '', regex=True)
df['Details'] = df['Details'].replace('<ul><li><ul><li>', '<ul><li>', regex=True)
df['Details'] = df['Details'].replace('</li></ul></li></ul>', '</li></ul>', regex=True)
df.head(50)

,48WS.com URL,Category,SKU #,Product Name,Image,Manufacturer_URL,Weight,Manufacturer,Description,Quick Overview,Details,Group Name,Info/Size/Color,UPC
0,nan,nan,9013,9013 Behind The Web Bracket,9013 (Updated Version).png,https://safewaze.com/,nan,Safewaze,The Safewaze 9013 dual leg bracket is designed...,nan,<ul><li>Zinc-Plated Steel</li><li>Forged Steel...,nan,Materials : Forged Steel / Aluminum Alloy; \nZ...,nan
1,nan,nan,018-4000,Chain Anchor,018-4000.png,https://safewaze.com/,nan,Safewaze,Designed for flat or sloped wood roof applicat...,nan,<ul><li>O-ring connection point</li><li>Durabl...,nan,Materials : Steel and Reinforced Synthetic\nWe...,nan
2,nan,nan,018-4001,Beam Trolley Anchor,018-4001.png,https://safewaze.com/,nan,Safewaze,The 018-4001 beam trolley anchor can be quickl...,nan,<ul><li> Lightweight Aluminum Body</li><li>Coa...,nan,"Materials : Aluminum, Stainless Steel, Ertalon...",nan
3,nan,nan,018-5007,7' Web Retractable with Aluminum Rebar Hook,018-5007.png,https://safewaze.com/,nan,Safewaze,The Safewaze 018-5007 single leg unit with alu...,nan,<ul><li>7' web</li><li>High strength clear pla...,nan,Material : High-strength clear plastic housing...,nan
4,nan,nan,018-5009,7' Web Retractable with Aluminum Snap Hook,018-5009 (With Carabiner).png,https://safewaze.com/,nan,Safewaze,The Safewaze 018-5009 single leg unit with an ...,nan,<ul><li>7' Web retractable</li><li>Aluminum sn...,nan,Material : High-strength clear plastic housing...,nan
5,nan,nan,018-5010,7' Web Retractable with Steel Rebar Hook,018-5010 (With Carabiner).png,https://safewaze.com/,nan,Safewaze,The Safewaze 018-5010 single leg unit with a s...,nan,<ul><li>7' web</li><li>Steel rebar hook</li><l...,nan,Materials : High-strength clear plastic housin...,nan
6,nan,nan,018-5025,7' Dual Leg Web Retractable with Aluminum Snap...,018-5025.png,https://safewaze.com/,nan,Safewaze,The Safewaze 018-5025 unit is lightweight with...,nan,<ul><li>7' web</li><li>Aluminum snap hooks</li...,nan,Materials : High-strength clear plastic housin...,nan
7,nan,nan,018-5026,7' Dual Leg Web Retractable with Steel Rebar H...,018-5026.png,https://safewaze.com/,nan,Safewaze,The Safewaze 018-5026 unit is lightweight with...,nan,<ul><li>7' web</li><li>Steel rebar hooks</li><...,nan,Materials : High-strength clear plastic housin...,nan
8,nan,nan,018-5027,7' Dual Leg Web Retractable with Aluminum Reba...,018-5027.png,https://safewaze.com/,nan,Safewaze,The Safewaze 018-5027 unit is lightweight with...,nan,<ul><li>7′ Web retractable</li><li>Aluminum re...,nan,Materials : High-strength clear plastic housin...,nan
9,nan,nan,018-5028,"7' Web 40"" Tie Back Retractable",018-5028.png,https://safewaze.com/,nan,Safewaze,The Safewaze 018-5028 tie back SRL comes with ...,nan,<ul><li>7′ length</li><li>Steel tie back snap ...,nan,Materials : High-strength clear plastic housin...,nan


In [859]:
df['Info/Size/Color'] = "<ul><li>" + df['Info/Size/Color'] + "</li></ul>"
df['Info/Size/Color'] = df['Info/Size/Color'].replace(r' \n', r'</li><li>', regex=True)
df['Info/Size/Color'] = df['Info/Size/Color'].replace(r'\n', r'</li><li>', regex=True)
df['Info/Size/Color'] = df['Info/Size/Color'].replace(r' \t', '</li><li>', regex=True)
df['Info/Size/Color'] = df['Info/Size/Color'].replace(r'\t', '</li><li>', regex=True)
df['Info/Size/Color'] = df['Info/Size/Color'].replace(r'<li> </li>', '', regex=True)
df['Info/Size/Color'] = df['Info/Size/Color'].replace(r'<li> </li>', '', regex=True)
df['Info/Size/Color'] = df['Info/Size/Color'].replace(r'<li></li>', '', regex=True)
df['Info/Size/Color'] = df['Info/Size/Color'].replace(r'<li></li>', '', regex=True)
df['Info/Size/Color'] = df['Info/Size/Color'].replace('<ul><li><ul><li>', '<ul><li>', regex=True)
df['Info/Size/Color'] = df['Info/Size/Color'].replace('</li></ul></li></ul>', '</li></ul>', regex=True)
df.head(50)

,48WS.com URL,Category,SKU #,Product Name,Image,Manufacturer_URL,Weight,Manufacturer,Description,Quick Overview,Details,Group Name,Info/Size/Color,UPC
0,nan,nan,9013,9013 Behind The Web Bracket,9013 (Updated Version).png,https://safewaze.com/,nan,Safewaze,The Safewaze 9013 dual leg bracket is designed...,nan,<ul><li>Zinc-Plated Steel</li><li>Forged Steel...,nan,<ul><li>Materials : Forged Steel / Aluminum Al...,nan
1,nan,nan,018-4000,Chain Anchor,018-4000.png,https://safewaze.com/,nan,Safewaze,Designed for flat or sloped wood roof applicat...,nan,<ul><li>O-ring connection point</li><li>Durabl...,nan,<ul><li>Materials : Steel and Reinforced Synth...,nan
2,nan,nan,018-4001,Beam Trolley Anchor,018-4001.png,https://safewaze.com/,nan,Safewaze,The 018-4001 beam trolley anchor can be quickl...,nan,<ul><li> Lightweight Aluminum Body</li><li>Coa...,nan,"<ul><li>Materials : Aluminum, Stainless Steel,...",nan
3,nan,nan,018-5007,7' Web Retractable with Aluminum Rebar Hook,018-5007.png,https://safewaze.com/,nan,Safewaze,The Safewaze 018-5007 single leg unit with alu...,nan,<ul><li>7' web</li><li>High strength clear pla...,nan,<ul><li>Material : High-strength clear plastic...,nan
4,nan,nan,018-5009,7' Web Retractable with Aluminum Snap Hook,018-5009 (With Carabiner).png,https://safewaze.com/,nan,Safewaze,The Safewaze 018-5009 single leg unit with an ...,nan,<ul><li>7' Web retractable</li><li>Aluminum sn...,nan,<ul><li>Material : High-strength clear plastic...,nan
5,nan,nan,018-5010,7' Web Retractable with Steel Rebar Hook,018-5010 (With Carabiner).png,https://safewaze.com/,nan,Safewaze,The Safewaze 018-5010 single leg unit with a s...,nan,<ul><li>7' web</li><li>Steel rebar hook</li><l...,nan,<ul><li>Materials : High-strength clear plasti...,nan
6,nan,nan,018-5025,7' Dual Leg Web Retractable with Aluminum Snap...,018-5025.png,https://safewaze.com/,nan,Safewaze,The Safewaze 018-5025 unit is lightweight with...,nan,<ul><li>7' web</li><li>Aluminum snap hooks</li...,nan,<ul><li>Materials : High-strength clear plasti...,nan
7,nan,nan,018-5026,7' Dual Leg Web Retractable with Steel Rebar H...,018-5026.png,https://safewaze.com/,nan,Safewaze,The Safewaze 018-5026 unit is lightweight with...,nan,<ul><li>7' web</li><li>Steel rebar hooks</li><...,nan,<ul><li>Materials : High-strength clear plasti...,nan
8,nan,nan,018-5027,7' Dual Leg Web Retractable with Aluminum Reba...,018-5027.png,https://safewaze.com/,nan,Safewaze,The Safewaze 018-5027 unit is lightweight with...,nan,<ul><li>7′ Web retractable</li><li>Aluminum re...,nan,<ul><li>Materials : High-strength clear plasti...,nan
9,nan,nan,018-5028,"7' Web 40"" Tie Back Retractable",018-5028.png,https://safewaze.com/,nan,Safewaze,The Safewaze 018-5028 tie back SRL comes with ...,nan,<ul><li>7′ length</li><li>Steel tie back snap ...,nan,<ul><li>Materials : High-strength clear plasti...,nan


In [860]:
df = df.replace('â€“', '-')
df = df.replace('â€²', '\"')
df = df.replace('â€™', '\"')
df = df.replace('â€³', '')
df = df.replace('â€','\"') # make sure you do this one after the others
df = df.replace('Â', '')
df = df.fillna('')
df = df.replace('^nan$', '', regex=True)
df.head(50)

,48WS.com URL,Category,SKU #,Product Name,Image,Manufacturer_URL,Weight,Manufacturer,Description,Quick Overview,Details,Group Name,Info/Size/Color,UPC
0,,,9013,9013 Behind The Web Bracket,9013 (Updated Version).png,https://safewaze.com/,,Safewaze,The Safewaze 9013 dual leg bracket is designed...,,<ul><li>Zinc-Plated Steel</li><li>Forged Steel...,,<ul><li>Materials : Forged Steel / Aluminum Al...,
1,,,018-4000,Chain Anchor,018-4000.png,https://safewaze.com/,,Safewaze,Designed for flat or sloped wood roof applicat...,,<ul><li>O-ring connection point</li><li>Durabl...,,<ul><li>Materials : Steel and Reinforced Synth...,
2,,,018-4001,Beam Trolley Anchor,018-4001.png,https://safewaze.com/,,Safewaze,The 018-4001 beam trolley anchor can be quickl...,,<ul><li> Lightweight Aluminum Body</li><li>Coa...,,"<ul><li>Materials : Aluminum, Stainless Steel,...",
3,,,018-5007,7' Web Retractable with Aluminum Rebar Hook,018-5007.png,https://safewaze.com/,,Safewaze,The Safewaze 018-5007 single leg unit with alu...,,<ul><li>7' web</li><li>High strength clear pla...,,<ul><li>Material : High-strength clear plastic...,
4,,,018-5009,7' Web Retractable with Aluminum Snap Hook,018-5009 (With Carabiner).png,https://safewaze.com/,,Safewaze,The Safewaze 018-5009 single leg unit with an ...,,<ul><li>7' Web retractable</li><li>Aluminum sn...,,<ul><li>Material : High-strength clear plastic...,
5,,,018-5010,7' Web Retractable with Steel Rebar Hook,018-5010 (With Carabiner).png,https://safewaze.com/,,Safewaze,The Safewaze 018-5010 single leg unit with a s...,,<ul><li>7' web</li><li>Steel rebar hook</li><l...,,<ul><li>Materials : High-strength clear plasti...,
6,,,018-5025,7' Dual Leg Web Retractable with Aluminum Snap...,018-5025.png,https://safewaze.com/,,Safewaze,The Safewaze 018-5025 unit is lightweight with...,,<ul><li>7' web</li><li>Aluminum snap hooks</li...,,<ul><li>Materials : High-strength clear plasti...,
7,,,018-5026,7' Dual Leg Web Retractable with Steel Rebar H...,018-5026.png,https://safewaze.com/,,Safewaze,The Safewaze 018-5026 unit is lightweight with...,,<ul><li>7' web</li><li>Steel rebar hooks</li><...,,<ul><li>Materials : High-strength clear plasti...,
8,,,018-5027,7' Dual Leg Web Retractable with Aluminum Reba...,018-5027.png,https://safewaze.com/,,Safewaze,The Safewaze 018-5027 unit is lightweight with...,,<ul><li>7′ Web retractable</li><li>Aluminum re...,,<ul><li>Materials : High-strength clear plasti...,
9,,,018-5028,"7' Web 40"" Tie Back Retractable",018-5028.png,https://safewaze.com/,,Safewaze,The Safewaze 018-5028 tie back SRL comes with ...,,<ul><li>7′ length</li><li>Steel tie back snap ...,,<ul><li>Materials : High-strength clear plasti...,


In [861]:
df.to_csv('Safewaze-Products-Sheet-03162021.csv')